In [6]:
import xarray as xr
import plotly as plty
import plotly.plotly as py
import plotly.offline as py_off
from plotly.graph_objs import *
import numpy as np
from scipy.io import netcdf



In [7]:
mapbox_access_token = 'pk.eyJ1IjoiamFja3AiLCJhIjoidGpzN0lXVSJ9.7YK6eRwUNFwd3ODZff6JvA'
plty.tools.set_credentials_file(username='__username__', api_key='__key__')

In [17]:
f_path = '/Downloads/avhrr-only-v2.20150101.nc'

from netCDF4 import Dataset

f = Dataset(f_path)
ds2 = xr.open_dataset(xr.backends.NetCDF4DataStore(f))



In [28]:
df2 = ds2.to_dataframe()
df_sub = df2
len(df_sub.index.get_level_values('lon')), len(df_sub['sst']) 

(1036800, 1036800)

In [29]:
step = 1.0
to_bin = lambda x: np.floor(x / step) * step
df_sub["latbin"] = df_sub.index.get_level_values('lat').map(to_bin)
df_sub["lonbin"] = df_sub.index.get_level_values('lon').map(to_bin)
groups = df_sub.groupby(("latbin", "lonbin"))


In [30]:
df_flat = df_sub.drop_duplicates(subset=['latbin', 'lonbin'])

df_no_nan = df_flat[np.isfinite(df_flat['sst'])]

len(df_no_nan)

df_no_nan.tail()

df_no_nan = df_no_nan[(df_no_nan.latbin < 76.0) & (df_no_nan.latbin > -66.0)]

colorscale = [[0, 'rgb(54, 50, 153)'], [0.35, 'rgb(17, 123, 215)'],
                [0.5, 'rgb(37, 180, 167)'], [0.6, 'rgb(134, 191, 118)'],
                [0.7, 'rgb(249, 210, 41)'], [1.0, 'rgb(244, 236, 21)']]

In [31]:
data = []

data.append(
    Scattermapbox(
        lon=df_no_nan['lonbin'].values,
        lat=df_no_nan['latbin'].values,
        mode='markers',
        marker=Marker(
            color=df_no_nan['sst'].values,
            colorscale=colorscale
        ),
    )
)
        
layout = Layout(
    margin=dict(t=0,b=0,r=0,l=0),
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=38,
            lon=-94
        ),
        pitch=0,
        zoom=0,
        style='light'
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Da_Weather_2.html')


/anaconda3/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:426: DeprecationWarning:

plotly.graph_objs.Marker is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Marker
  - plotly.graph_objs.histogram.selected.Marker
  - etc.




High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~wkwww/0 or inside your plot.ly account where it is named 'Da_Weather_2.html'
